In [1]:
# dimension_choice.py
import random

# ALL_ARGS = ["datapoints", "nbr_dims", "last_dim"]


def alternate_dim(nbr_dims, last_dim, **kwargs):
    """
    Returns the next dimension to split on.
    """
    return {"dim": (last_dim + 1) % nbr_dims}

ALTERNATE_OUT_PLUS = []


def random_dim(nbr_dims, **kwargs):
    """
    Returns a random dimension to split on.
    """
    return {"dim": random.randint(0, nbr_dims - 1)}

RANDOM_OUT_PLUS = []


def max_variance_dim(datapoints: list[list[float]], nbr_dims: int, **kwargs):
    """
    Returns the dimension with the highest variance.
    """
    max_variance = 0
    max_variance_dim = 0

    mean_val = 0

    mean_val_out = 0

    for dim in range(nbr_dims):
        variance = 0
        # calculate the mean value across all points for the current dimension
        mean_val = sum([point[dim] for point in datapoints]) / len(datapoints)

        # calculate the variance
        variance = sum([(point[dim] - mean_val) ** 2 for point in datapoints])
        variance /= len(datapoints)

        if variance > max_variance:
            max_variance = variance
            max_variance_dim = dim
            mean_val_out = mean_val
    return {"dim": max_variance_dim, "mean_val": mean_val_out}

MAX_VARIANCE_OUT_PLUS = ["mean_val"]


def widest_interval_dim(datapoints: list[list[float]], nbr_dims: int, **kwargs):
    """
    Returns the dimension with the highest maximum-minimum value.
    """
    max_range = 0
    max_range_dim = 0

    max_val = 0
    min_val = 0

    for dim in range(nbr_dims):
        max_val = max([point[dim] for point in datapoints])
        min_val = min([point[dim] for point in datapoints])
        range_val = max_val - min_val
        if range_val > max_range:
            max_range = range_val
            max_range_dim = dim
    return {"dim": max_range_dim, "max_val": max_val, "min_val": min_val}

WIDEST_INTERVAL_OUT_PLUS = ["max_val", "min_val"]

DIM_OUT_PLUS = {
    "alternate": ALTERNATE_OUT_PLUS,
    "random": RANDOM_OUT_PLUS,
    "max_variance": MAX_VARIANCE_OUT_PLUS,
    "widest_interval": WIDEST_INTERVAL_OUT_PLUS,
}


In [2]:
# split_position_choice.py
import random

# ALL_ARGS = ["datapoints", "dim", "sort", "length"]


def mean_split(
    datapoints: list[list[float]], dim: int, mean_val: float = None, **kwargs
):
    """
    Returns the mean value of the given dimension.
    """

    if mean_val is not None:
        return mean_val
    return sum([point[dim] for point in datapoints]) / len(datapoints)


def median_split(
    datapoints: list[list[float]],
    dim: int,
    sort: bool = False,
    length: int = None,
    **kwargs
):
    """
    Returns the median value of the given dimension.
    """
    if not sort:
        sorted_points = sorted(datapoints, key=lambda x: x[dim])
        mid = len(sorted_points) // 2
    else:
        sorted_points = datapoints
        mid = length // 2
    return sorted_points[mid][dim]


def random_split(
    datapoints: list[list[float]], dim: int, sort: bool = False, **kwargs
):
    """
    Returns a random value of the given dimension.
    """

    if not sort:
        sorted_points = sorted(datapoints, key=lambda x: x[dim])
    else:
        sorted_points = datapoints
    split = (
        random.random() * (sorted_points[-1][dim] - sorted_points[0][dim])
        + sorted_points[0][dim]
    )
    return split


def geometric_center_split(
    datapoints: list[list[float]],
    dim: int,
    sort: bool = False,
    max_val: float = None,
    min_val: float = None,
    **kwargs
):
    """
    Returns the geometric center of the given dimension.
    """

    if max_val is not None and min_val is not None:
        return (max_val + min_val) / 2
    if not sort:
        sorted_points = sorted(datapoints, key=lambda x: x[dim])
    else:
        sorted_points = datapoints
    return (sorted_points[0][dim] + sorted_points[-1][dim]) / 2


In [3]:
# from dimension_choice import *
# from split_position_choice import *
from sklearn.metrics import silhouette_score
import numpy as np

class KDTree:
    """
    A class to represent a KDTree.

    Attributes
    ----------
    k : int
        The number of dimensions of the datapoints.
    root : dict
        The root node of the KDTree.
    dimension_choice : function
        The function to choose the dimension to split on.
    split_position_choice : function
        The function to choose the split position.
    leaf_size : int
        The maximum number of points that can be stored in a leaf node.
    max_depth : int
        The maximum depth of the tree.

    Methods
    -------
    build(datapoints: list[list[float]]) -> dict
        Builds the KDTree from the given datapoints.
    recursive_build(datapoints: list[list[float]], depth: int) -> dict
        Recursively builds the KDTree from the given datapoints.
    compute_silhouette_score() -> float
        Computes the Silhouette Score for the KDTree.
    """

    def __init__(
        self,
        k: int,
        datapoints: list[list[float]],
        dimension_choice: str = "random",
        split_position_choice: str = "random",
        leaf_size: int = 10,
        max_depth: int = None,
    ):
        """
        Initializes the KDTree with the given datapoints and the dimension_choice and split_position_choice functions.

        Parameters
        ----------
        datapoints : list[list[float]]
            The list of datapoints to build the KDTree from.
        dimension_choice : str, optional
            The function to choose the dimension to split on, by default "random".
            Options: "alternate", "random", "max_variance", "widest_interval".
        split_position_choice : str, optional
            The function to choose the split position, by default "random".
            Options: "mean", "median", "random", "geometric_center".
        leaf_size : int, optional
            The maximum number of points that can be stored in a leaf node, by default 10.
        max_depth : int, optional
            The maximum depth of the tree, by default None.

        Raises
        ------
        ValueError
            If the dimension_choice or split_position_choice is not a valid function.
        """

        self.k = k
        self.leaf_size = leaf_size
        self.max_depth = max_depth

        switcher: dict[str, function] = {
            "alternate": alternate_dim,
            "random": random_dim,
            "max_variance": max_variance_dim,
            "widest_interval": widest_interval_dim,
        }

        assert dimension_choice in switcher, "Invalid dimension_choice, choose from 'alternate', 'random', 'max_variance', 'widest_interval'"

        self.dimension_choice = switcher[dimension_choice]

        self.dim_out_plus = DIM_OUT_PLUS[dimension_choice]

        switcher = {
            "mean": mean_split,
            "median": median_split,
            "random": random_split,
            "geometric_center": geometric_center_split,
        }

        assert split_position_choice in switcher, "Invalid split_position_choice, choose from 'mean', 'median', 'random', 'geometric_center'"

        self.split_position_choice = switcher[split_position_choice]

        self.root = self.build(datapoints)

    def build(self, datapoints):
        if len(datapoints) == 0:
            return None
        return self.recursive_build(datapoints, 0)

    def recursive_build(self, datapoints: list[list[float]], depth: int, last_dim: int = 0):
        # Stop recursion if the number of points is <= leaf_size or max_depth is reached
        if len(datapoints) <= self.leaf_size or (self.max_depth is not None and depth >= self.max_depth):
            return {
                "depth": depth,
                "points": datapoints,
                "leaf": True,
            }

        kwargs = {"datapoints": datapoints, "nbr_dims": self.k, "last_dim": last_dim}
        dim_result = self.dimension_choice(**kwargs)
        split_dim = dim_result["dim"]
        plus = {key: dim_result[key] for key in self.dim_out_plus}

        kwargs = {"datapoints": datapoints, "dim": split_dim}

        split_result = self.split_position_choice(**kwargs, **plus)
        split_val = split_result

        sorted_points = sorted(datapoints, key=lambda x: x[split_dim])

        left_points = []
        right_points = []

        for point in sorted_points:
            if point[split_dim] < split_val:
                left_points.append(point)
            else:
                right_points.append(point)

        if len(left_points) == 0:
            left_child = None
        else:
            left_child = self.recursive_build(left_points, depth + 1, split_dim)

        if len(right_points) == 0:
            right_child = None
        else:
            right_child = self.recursive_build(right_points, depth + 1, split_dim)

        return {
            "split_dim": split_dim,
            "split_val": split_val,
            "left": left_child,
            "right": right_child,
            "depth": depth,
            "leaf": False,
        }

    def compute_silhouette_score(self):
        """
        Computes the Silhouette Score for the KDTree.

        Returns
        -------
        float
            The Silhouette Score of the KDTree.
        """
        # Flatten the tree to get all points and their cluster labels
        points, labels = self._flatten_tree(self.root, 0)
        points = np.array(points)
        labels = np.array(labels)

        # Compute the Silhouette Score
        score = silhouette_score(points, labels)
        return score

    def _flatten_tree(self, node, label):
        """
        Flattens the KDTree to get all points and their cluster labels.

        Parameters
        ----------
        node : dict
            The current node of the KDTree.
        label : int
            The current cluster label.

        Returns
        -------
        list[list[float]], list[int]
            The list of points and their cluster labels.
        """
        if node is None:
            return [], []

        if node["leaf"]:
            points = node["points"]
            labels = [label] * len(points)
            return points, labels

        left_points, left_labels = self._flatten_tree(node["left"], label)
        right_points, right_labels = self._flatten_tree(node["right"], label + 1)

        return left_points + right_points, left_labels + right_labels

In [4]:
# import sys

# sys.path.append('.')
# sys.path.append('./kd_tree/')

# from kd_tree.kd_tree import KDTree
import time
import sys
import tracemalloc
import h5py

In [5]:
train = []

with h5py.File('/content/drive/MyDrive/db2/gist-960-euclidean.hdf5', 'r') as f:
    print("Keys: %s" % f.keys())
    # Get the data
    train = f['train'][:]

    test = f['test'][:] 
    distances = f['distances'][:] 
    neighbors = f['neighbors'][:] 


Keys: <KeysViewHDF5 ['distances', 'neighbors', 'test', 'train']>


In [6]:
variants = [
    {"dimension_choice": "random",          "split_position_choice": "random",            "leaf_size": 10, "max_depth": None},
    {"dimension_choice": "max_variance",    "split_position_choice": "mean",              "leaf_size": 20, "max_depth": 20},
    {"dimension_choice": "widest_interval", "split_position_choice": "median",            "leaf_size": 30, "max_depth": 15},
    {"dimension_choice": "alternate",       "split_position_choice": "geometric_center",  "leaf_size": 40, "max_depth": 10},
    {"dimension_choice": "random",          "split_position_choice": "mean",              "leaf_size": 50, "max_depth": 5},
    {"dimension_choice": "max_variance",    "split_position_choice": "median",            "leaf_size": 60, "max_depth": 25},
    {"dimension_choice": "widest_interval", "split_position_choice": "geometric_center",  "leaf_size": 70, "max_depth": 30},
    {"dimension_choice": "alternate",       "split_position_choice": "mean",              "leaf_size": 80, "max_depth": 35},
    {"dimension_choice": "random",          "split_position_choice": "median",            "leaf_size": 90, "max_depth": 40},
    {"dimension_choice": "max_variance",    "split_position_choice": "geometric_center",  "leaf_size": 100, "max_depth": 45},

]

In [7]:
results = []

for variant in variants:
    print(f"Running experiment with variant: {variant}")

    # Start tracking memory usage
    tracemalloc.start()

    # Measure the time to build the index
    start_time = time.time()

    tree = KDTree(
        k=960,
        datapoints=train[:10000],  # Use a subset of the data for faster experiments
        dimension_choice=variant["dimension_choice"],
        split_position_choice=variant["split_position_choice"],
        leaf_size=variant["leaf_size"],
        max_depth=variant["max_depth"]
    )

    build_time = time.time() - start_time

    # Measure the memory usage
    current_memory, peak_memory = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    score = tree.compute_silhouette_score()

    results.append({
        "variant": variant,
        "build_time": build_time,
        "peak_memory": peak_memory,
        "silhouette_score": score
    })

    print(f"Build Time: {build_time:.2f} seconds")
    print(f"Peak Memory: {peak_memory / 1024 / 1024:.2f} MB")
    print(f"Silhouette Score: {score:.4f}")
    print("-" * 40)



Running experiment with variant: {'dimension_choice': 'random', 'split_position_choice': 'random', 'leaf_size': 10, 'max_depth': None}
Build Time: 1.53 seconds
Peak Memory: 3.96 MB
Silhouette Score: -0.0838
----------------------------------------
Running experiment with variant: {'dimension_choice': 'max_variance', 'split_position_choice': 'mean', 'leaf_size': 20, 'max_depth': 20}
Build Time: 533.77 seconds
Peak Memory: 2.49 MB
Silhouette Score: -0.0782
----------------------------------------
Running experiment with variant: {'dimension_choice': 'widest_interval', 'split_position_choice': 'median', 'leaf_size': 30, 'max_depth': 15}
Build Time: 198.57 seconds
Peak Memory: 2.29 MB
Silhouette Score: -0.0754
----------------------------------------
Running experiment with variant: {'dimension_choice': 'alternate', 'split_position_choice': 'geometric_center', 'leaf_size': 40, 'max_depth': 10}
Build Time: 0.86 seconds
Peak Memory: 2.82 MB
Silhouette Score: 0.0879
--------------------------

In [8]:
results

[{'variant': {'dimension_choice': 'random',
   'split_position_choice': 'random',
   'leaf_size': 10,
   'max_depth': None},
  'build_time': 1.5347061157226562,
  'peak_memory': 4156522,
  'silhouette_score': -0.08379764},
 {'variant': {'dimension_choice': 'max_variance',
   'split_position_choice': 'mean',
   'leaf_size': 20,
   'max_depth': 20},
  'build_time': 533.7726862430573,
  'peak_memory': 2614582,
  'silhouette_score': -0.07822317},
 {'variant': {'dimension_choice': 'widest_interval',
   'split_position_choice': 'median',
   'leaf_size': 30,
   'max_depth': 15},
  'build_time': 198.5659167766571,
  'peak_memory': 2406144,
  'silhouette_score': -0.075402856},
 {'variant': {'dimension_choice': 'alternate',
   'split_position_choice': 'geometric_center',
   'leaf_size': 40,
   'max_depth': 10},
  'build_time': 0.8612453937530518,
  'peak_memory': 2959172,
  'silhouette_score': 0.08787973},
 {'variant': {'dimension_choice': 'random',
   'split_position_choice': 'mean',
   'leaf_s